# Dungeons and Dragons Dataset Analysis

## Setting up 

Here we want to set up all the packages we want to work with.

In [1]:
import requests
import json
import pandas as pd

## Now to get the data.
We are using a dataset provided as a json from https://github.com/oganm/dnddata

Since the dataset is quite large (more than 7000 unique characters as of february 2022) we will first download it and save it locally. This way, we don't have to download the data again every time we run an analysis unless we want to update the data.

In [2]:

# Download the json from https://github.com/oganm/dnddata

url = "https://raw.githubusercontent.com/oganm/dnddata/master/data-raw/dnd_chars_unique.json"

with requests.get(url) as content:
    json_data = content.text

data = json.loads(json_data)

# Save the json file locally

with open("json_data.json", "w") as f:
    f.writelines(json.dumps(data, indent=2))

Next, we load the .json file we just downloaded.

In [3]:
# open local json file

with open("json_data.json", "r") as f:
    data = json.load(f)

The next step is to go through the .json file and create a table from the data. Here we select which attributes are interesting and which ones we want to leave out.

Finally, we create a dataframe from the data.

At this point, we could continue the analysis using python if we want.

In [6]:
# make a Data Frame
data_dict = {
    "Name": [],
    "Race": [],
    "Class": [],
    "Total_lvl": [],
    "Alignment": [],
    "Skills": [],
    "Feats": [],
    "HP": [],
    "AC": [],
    "Str": [], "Dex": [], "Con": [], "Int": [], "Wis": [], "Cha": [],
    "Spellcaster": [],
    "Num_of_spells": []
}

# loop through the characters in the json and pick out the attributes for analysis
for char in data:
    # name
    char_name = data[char]["name"]["alias"][0]
    data_dict["Name"].append(char_name)

    #race
    char_race = data[char]["race"]["processedRace"][0]
    data_dict["Race"].append(char_race)

    # class
    class_str = ""
    for cls in data[char]["class"]:
        if class_str != "":
            class_str = class_str + "; "
        this_class = data[char]["class"][cls]["class"][0]
        subclass = data[char]["class"][cls]["subclass"][0]
        if subclass == "":
            subclass = "No Sub"
        class_lvl = data[char]["class"][cls]["level"][0]
        this_class_str = f"[{this_class}; {subclass}; {class_lvl}]"
        class_str = class_str + this_class_str
    data_dict["Class"].append(class_str)

    # character level
    total_lvl = data[char]["level"][0]
    data_dict["Total_lvl"].append(total_lvl)
    
    # alignment
    if data[char]["alignment"]["processedAlignment"][0]:
        alignment = data[char]["alignment"]["processedAlignment"][0]
    else:
        alignment = None
    data_dict["Alignment"].append(alignment)

    # skills / proficiencies
    skill_str = ""
    for skill in data[char]["skills"]:
        if skill_str != "":
            skill_str = skill_str + "; "
        skill_str = skill_str + skill
    data_dict["Skills"].append(skill_str)

    # feats
    feat_str = ""
    for feat in data[char]["feats"]:
        if feat_str != "":
            feat_str = feat_str + "; "
        feat_str = feat_str + feat
    data_dict["Feats"].append(feat_str)

    # HP 
    hp = data[char]["HP"][0]
    data_dict["HP"].append(hp)

    # AC
    ac = data[char]["AC"][0]
    data_dict["AC"].append(ac)

    # attributes
    for atr in ["Str", "Dex", "Con", "Int", "Wis", "Cha"]:
        val = data[char]["attributes"][atr][0]
        data_dict[atr].append(val)

    # spellcaster
    if data[char]["spells"]:
        data_dict["Spellcaster"].append(True)
    else:
        data_dict["Spellcaster"].append(False)

    # number of spells
    data_dict["Num_of_spells"].append(len(data[char]["spells"]))


df = pd.DataFrame(data=data_dict)

Now we can export the data frame we created as a .csv to use for further analysis using other tools.

In [7]:
df.to_csv("dnd_data.csv")